In [ ]:
import sklearn
import numpy as np
import pandas as pd


In [ ]:
data=pd.read_csv('/content/DataPreprocessingGraded_dataset.csv')

In [ ]:
data.head()

,V1,V2,V3,V4,V5,Target
0,2.0,50.0,12500.0,98.0,NEGATIVE,YES
1,0.0,13.0,3250.0,28.0,NEGATIVE,YES
2,?,?,4000.0,35.0,NEGATIVE,YES
3,?,20.0,5000.0,45.0,NEGATIVE,YES
4,1.0,24.0,6000.0,77.0,NEGATIVE,NO


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      748 non-null    object 
 1   V2      748 non-null    object 
 2   V3      748 non-null    float64
 3   V4      748 non-null    float64
 4   V5      748 non-null    object 
 5   Target  748 non-null    object 
dtypes: float64(2), object(4)
memory usage: 35.2+ KB


In [ ]:
feature_matrix=data.drop('Target',axis=1)
feature_matrix.head()

,V1,V2,V3,V4,V5
0,2.0,50.0,12500.0,98.0,NEGATIVE
1,0.0,13.0,3250.0,28.0,NEGATIVE
2,?,?,4000.0,35.0,NEGATIVE
3,?,20.0,5000.0,45.0,NEGATIVE
4,1.0,24.0,6000.0,77.0,NEGATIVE


In [ ]:
target_matrix=data['Target']
target_matrix.head()

0    YES
1    YES
2    YES
3    YES
4     NO
Name: Target, dtype: object

In [ ]:
feature_matrix=feature_matrix.replace('?',np.nan)
feature_matrix.head()

,V1,V2,V3,V4,V5
0,2.0,50.0,12500.0,98.0,NEGATIVE
1,0.0,13.0,3250.0,28.0,NEGATIVE
2,NaN,NaN,4000.0,35.0,NEGATIVE
3,NaN,20.0,5000.0,45.0,NEGATIVE
4,1.0,24.0,6000.0,77.0,NEGATIVE


In [ ]:
feature_matrix['V1']=feature_matrix['V1'].astype('float')
feature_matrix['V2']=feature_matrix['V2'].astype('float')

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline

In [ ]:
si=SimpleImputer(strategy='mean')
ss=StandardScaler()
oe=OrdinalEncoder()

In [ ]:
from sklearn.compose import make_column_transformer

In [ ]:
num_pipeline=Pipeline(
    steps=[('simple_imputer',SimpleImputer(missing_values=np.nan)),
           ('standard_scaler',StandardScaler())]
)

In [ ]:
cat_pipeline=Pipeline(
    steps=[('ordinal_encoder',OrdinalEncoder())]
)

In [ ]:
from sklearn.compose import ColumnTransformer

In [ ]:
full_pipeline=ColumnTransformer(
    [('numerical',num_pipeline,['V1','V2','V3','V4']),
     ('categorical',cat_pipeline,['V5'])]
)

In [ ]:
data_new=full_pipeline.fit_transform(feature_matrix)

In [ ]:
from sklearn.feature_selection import VarianceThreshold
vi=VarianceThreshold(threshold=0.1)

In [ ]:
final_data=vi.fit_transform(data_new)

In [ ]:
final_data[:5]

array([[-9.38169390e-01,  7.70986653e+00,  7.62334626e+00,
         2.61563344e+00],
       [-1.18627754e+00,  1.30454949e+00,  1.28273826e+00,
        -2.57880900e-01],
       [ 0.00000000e+00, -1.53758496e-16,  1.79684161e+00,
         2.94705348e-02],
       [ 0.00000000e+00,  2.51636623e+00,  2.48231275e+00,
         4.39972584e-01],
       [-1.06222347e+00,  3.20883294e+00,  3.16778388e+00,
         1.75357914e+00]])

In [ ]:
from sklearn.feature_selection import RFE

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
rfe=RFE(estimator=LogisticRegression())

In [ ]:
rfe_data=rfe.fit_transform(final_data,target_matrix)

In [ ]:
rfe_data[:5]

array([[-0.93816939,  7.62334626],
       [-1.18627754,  1.28273826],
       [ 0.        ,  1.79684161],
       [ 0.        ,  2.48231275],
       [-1.06222347,  3.16778388]])

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector

In [ ]:
forwaed_sfs=SequentialFeatureSelector(estimator=LogisticRegression(), n_features_to_select=2, tol=None, direction='forward')

In [ ]:
sfs_forward_data=forwaed_sfs.fit_transform(final_data,target_matrix)

In [ ]:
sfs_forward_data[:5]
#compare with the final_data which features are remained

array([[ 7.70986653e+00,  2.61563344e+00],
       [ 1.30454949e+00, -2.57880900e-01],
       [-1.53758496e-16,  2.94705348e-02],
       [ 2.51636623e+00,  4.39972584e-01],
       [ 3.20883294e+00,  1.75357914e+00]])

In [ ]:
backward_sfs=SequentialFeatureSelector(estimator=LogisticRegression(), n_features_to_select=2, tol=None, direction='backward')

In [ ]:
sfs_backward_data=backward_sfs.fit_transform(final_data,target_matrix)

In [ ]:
sfs_backward_data[:5]
#compare with the final_data which features are remained

array([[ 7.62334626,  2.61563344],
       [ 1.28273826, -0.2578809 ],
       [ 1.79684161,  0.02947053],
       [ 2.48231275,  0.43997258],
       [ 3.16778388,  1.75357914]])